In [8]:
!pip install pdfplumber
!brew install freetype imagemagick
!brew install ghostscript

To reinstall 2.10.1, run `brew reinstall freetype`
To reinstall 7.0.9-26, run `brew reinstall imagemagick`
==> Downloading https://homebrew.bintray.com/bottles/ghostscript-9.50.catalina.b
==> Downloading from https://akamai.bintray.com/c1/c1e11a68fdd8b406979fc51791cb1
######################################################################## 100.0%
==> Pouring ghostscript-9.50.catalina.bottle.tar.gz
🍺  /usr/local/Cellar/ghostscript/9.50: 670 files, 82.3MB


In [ ]:
# 필요 데이터

# 진로 희망사항 ->  

# 창의적 체험활동
    #1학년
        #자율활동
        #동아리 활동
        #봉사 활동
        #진로 활동
    #2학년
        #자율활동
        #동아리 활동
        #봉사 활동
        #진로 활동
    #3학년
        #자율활동
        #동아리 활동
        #봉사 활동
        #진로 활동

# 세부 특기사항

# 봉사활동




data = {
    "진로희망사항": {
        "1학년":{
            "진로 희망": "",
            "희망 사유": "",
        },
        "2학년":{
            "진로 희망": "",
            "희망 사유": "",
        }
    },
    
    "창의적체험활동": {
        "1학년": {
            
            "자율활동":{

            },
            "동아리 활동":{

            },
            "봉사 활동":{

            },
            "진로 활동":{

            },
        },
        "2학년": {
            "자율활동":{

            },
            "동아리 활동":{

            },
            "봉사 활동":{

            },
            "진로 활동":{

            },
        },
        "3학년": {
            "자율활동":{

            },
            "동아리 활동":{

            },
            "봉사 활동":{

            },
            "진로 활동":{

            },
        },
    },
}


        

In [1544]:
import pdfplumber

    #first_page = pdf.pages[2]
       
    # 추출 이미지 확인
    #im = first_page.to_image(resolution=150)
    #im.draw_rects(first_page.extract_())
    #im.save("./output", format="PNG")

def aranngeList(data, start, end):
    del data[start:end]
    return data    

"""

"""

def getRawDataSetPage(file, index):
    with pdfplumber.open(file) as pdf:
        targetPage = pdf.pages[index]
        return targetPage.extract_tables()

def getRawData(file):
    with pdfplumber.open(file) as pdf:

        # 데이터 범위 기반 요소 인덱싱
        


        봉사실적 = [] 
        창의적체험활동상황 = [] 
        진로희망사항 = [] 
        세부특기사항 = []
        수상실적 = [] 
        과목 = ['현장실습유형', '현장실습 직무분야', '실습기간', '실습내용']


        
        
        # 생기부 버전 읽기
        try:
            pageVersion = int(pdf.pages[0].extract_tables()[-2][0][1][:4])
        except:
            pageVersion = int(pdf.pages[0].extract_tables()[-1][0][1][:4])

        print(pageVersion)


        for pageIndex in range(len(pdf.pages)):
            targetPage = pdf.pages[pageIndex]
            table = targetPage.extract_tables()


            for dataSets in table:

                if len(dataSets[0]) > 0:
                    if dataSets[0][0] != None:
                        flagData = dataSets[0][0].replace(" ", "") #2020년 생기부 데이터 기준 파싱 위치
                        if flagData[2:] == '세부능력및특기사항':
                            세부특기사항.append(dataSets[1])

                        # ==== 수상경력 데이터 파싱 부분 ====

                        if flagData[2:] == '수상경력': # 해당 부분은 2016버전, 2020버전 모두 동작함.
                            [수상실적.append(index) for index in dataSets[2:]]

                        if flagData == '수상명': # 2020 버전만 동작함
                            [수상실적.append(index) for index in dataSets[1:]]

                        # ==== 진로희망사항 데이터 파싱 부분 ====

                        if flagData[2:] == '진로희망사항' and pageVersion <= 2016: # 2016년 이하 버전에서만 동작
                            [진로희망사항.append([index[0], index[3], index[4]]) for index in dataSets[3:]]



                if len(dataSets[0]) > 1:
                    if dataSets[0][1] != None:
                        flagData = dataSets[0][1].replace(" ", "")
                        # ==== 창의적체험활동상황 데이터 파싱 부분 ====
                        
                        if  flagData == '창의적체험활동상황':              
                            [창의적체험활동상황.append(index) for index in dataSets[2:]]

                        # ==== 봉사활동실적 데이터 파싱 부분 ====

                        if flagData == '봉사활동실적':
                            [봉사실적.append(index[0:-1]) for index in dataSets[2:]]



                if len(dataSets) > 1:
                    if len(dataSets[0]) > 1 :
                        if dataSets[1][1] != None: 
                            flagData = dataSets[1][1].replace(" ", "")
                            # ==== 진로희망 데이터 파싱 부분 ====
                            if flagData == '진로희망': #2020년 버전만 동작함, 처음 항목 파싱
                                [진로희망사항.append(index) for index in dataSets[2:]]
                            
                            # ==== 과목 데이터 파싱(예체능 과목) ====
                            if flagData == '과목': # 2016, 2020년 버전 동작, 예체능 데이터 추출
                                for index in dataSets[2:]:
                                    if index[1] != None:
                                        과목.append(index[1].replace("\n", ""))

                            # =============================

                        if dataSets[0][1] != None:
                            flagData = dataSets[0][1].replace(" ", "")
                            
                            #==== 과목 데이터 파싱(일반교과 과목) ====
                            if flagData  == '과목': # 2016, 2020년 버전 동작, 일반교과 데이터 추출
                                for index in dataSets[1:]:
                                    if index[1] != None:
                                        과목.append(index[1].replace("\n", ""))
                            
                            
                            
                            # ==== 진로희망 데이터 파싱 부분 ====

                            if flagData  == '진로희망': #2020년 버전만 동작
                                [진로희망사항.append(index) for index in dataSets[2:]]

                            # ==== 생기부 수상경력 데이터 파싱 부분 ====

                            if flagData == '수상명': # 2016 버전만 동작함, index 0번 데이터가 비어있음.
                                [수상실적.append(index) for index in dataSets[1:]]

                            # ==== 진로희망 데이터 파싱 부분 ====

                            if flagData == '특기또는흥미': # 2016 버전만 동작함. 편의를 위해 특기또는흥미로 검색하도록 설정함
                                [진로희망사항.append(aranngeList(index, 1, 3)) for index in dataSets[2:]]
                                
    return [봉사실적, 창의적체험활동상황, 진로희망사항, 세부특기사항, 수상실적, 과목]
    
    # 봉사실적 1, 3,
    # 창의적체험활동상황 1, 3
    # 진로희망사항 1, 3
    # 세부특기사항 1, 3
    # 수상실적 1, 3
    
    #for i in 진로희망사항:
    #    print(i)
        
    #print(진로희망사항)   



    
def dataNormalizationEmptyData(rawData, appendLength = 1): # 봉사실적, 수상 실적, 진로희망사항
    for cursor in range(len(rawData) - 1):
        for index in range(appendLength):
            if rawData[cursor][index] != None:
                flagData = rawData[cursor][index]

            if rawData[cursor + 1][index] == None or rawData[cursor + 1][index] == '':
                rawData[cursor + 1][index] = flagData
    
    return rawData
    
def dataNormalization_봉사실적(rawData):
    for index in range(len(rawData)):
        if rawData[index][2] == '-':
            rawData[index][1] += ' - ' + rawData[index][3]
            
        rawData[index] = aranngeList(rawData[index], 2, 4)
        
        
    return dataNormalizationEmptyData(rawData)
        

def dataNormalization_창의적체험활동상황(rawData):
    return dataNormalizationEmptyData(rawData, 2)

def dataNormalization_진로희망사항(rawData):
    initData = [[], [], []]
    rawData = dataNormalizationEmptyData(rawData, 1)
  
    for index in range(len(rawData)):
        if rawData[index][1] == '':
            initData[int(rawData[index - 1][0]) - 1][2] += rawData[index][2]
            
        else:
            initData[int(rawData[index][0]) - 1] = rawData[index]
    
    return initData



def dataNormalization_수상실적(rawData):
    return dataNormalizationEmptyData(rawData)


        
def formatData_Json_진로희망사항(raw_data_list):
    formatedData = {
       "1학년":{
            "진로희망": "",
            "희망사유": "",
        },
        "2학년":{
            "진로희망": "",
            "희망사유": "",
        } ,
        "3학년":{
            "진로희망": "",
            "희망사유": "",
        } ,
    } 
    
    raw_data_list = dataNormalization_진로희망사항(raw_data_list)
    
    for dataSet in raw_data_list:
        if len(dataSet) > 2: # 데이터가 모두 들어있을때만 입력
            formatedData[dataSet[0] + "학년"]= {
                "진로희망": dataSet[1].replace("\n", ""),
                "희망사유": dataSet[2].replace("\n", "")
            }
    return formatedData


def formatData_Json_봉사실적(raw_data_list):
    initData = [[], [], []]
    
    raw_data_list = dataNormalization_봉사실적(raw_data_list)
    
    for index in range(len(raw_data_list)):
        dataSet = raw_data_list[index]
        initData[int(dataSet[0]) - 1].append(
            {
                "날짜": dataSet[1].replace("\n", ""),
                "기관": dataSet[2].replace("\n", ""),
                "활동내용": dataSet[3].replace("\n", ""),
                "봉사시간": dataSet[4].replace("\n", ""),
            }
        )
        
    formatedData = {
        "1학년": {},
        "2학년": {},
        "3학년": {},
    }
        
    for gradeIndex in range(len(initData)):
        dataSets = initData[gradeIndex]
        for index in range(len(dataSets)):
            formatedData[str(gradeIndex + 1)  + "학년"][index] = dataSets[index]

    return formatedData


def formatData_Json_수상실적(raw_data_list):
    formatedData = {} 
    
    raw_data_list = dataNormalization_수상실적(raw_data_list)
    
    for index in range(len(raw_data_list)):
        dataSet = raw_data_list[index]
        formatedData[index] = {
            "구분": dataSet[1].replace("\n", ""),
            "수상명": dataSet[2].replace("\n", ""),
            "등급": dataSet[3].replace("\n", ""),
            "수여기관": dataSet[4].replace("\n", ""),
            "참가대상": dataSet[4].replace("\n", ""),
        }
               
    return formatedData

def formatData_Json_세부특기사항(raw_data_list):
    raw_data_list = dataNormalization_세부특기사항(raw_data_list)
    formatedData = {}
    dataIndex = ''
    
    for dataSets in raw_data_list:
        if len(dataSets) > 1:
            if dataIndex != '': formatedData[dataIndex] += '.'
            
            dataIndex = dataSets[0].replace("\n", "").strip()
            formatedData[dataIndex] = dataSets[1].replace("\n", "")
            
        else:
            formatedData[dataIndex] += "." + dataSets[0].replace("\n", "")
    
    return formatedData



def formatData_Json_창의적체험활동상황(raw_data_list):
    formatedData = {
        "1학년": {
            "자율활동":{
                "시간": "",
                "내용": "",
            },
            "동아리활동": {
                "시간": "",
                "내용": "",
            },
            "봉사활동": {
                "시간": "",
                "내용": "",
            },
            "진로활동": {
                "시간": "",
                "내용": "",
            }
        },
        "2학년": {
            "자율활동":{
                "시간": "",
                "내용": "",
            },
            "동아리활동": {
                "시간": "",
                "내용": "",
            },
            "봉사활동": {
                "시간": "",
                "내용": "",
            },
            "진로활동": {
                "시간": "",
                "내용": "",
            }
        },
        "3학년": {
            "자율활동":{
                "시간": "",
                "내용": "",
            },
            "동아리활동": {
                "시간": "",
                "내용": "",
            },
            "봉사활동": {
                "시간": "",
                "내용": "",
            },
            "진로활동": {
                "시간": "",
                "내용": "",
            }
        }
    }

    raw_data_list = dataNormalization_창의적체험활동상황(raw_data_list)

    for dataSet in raw_data_list:
        formatedData[str(dataSet[0]) + "학년"][dataSet[1]]["시간"] += dataSet[2]
        formatedData[str(dataSet[0]) + "학년"][dataSet[1]]["내용"] += dataSet[3].replace("\n", "")
    
    return formatedData
  
    


def dataNormalization_세부특기사항(rawData):
    rawDataString = ""
    initData = []
    splitedData = []

    for data in rawData: rawDataString += data[0] #init rawDataString
    
    for data_step1 in rawDataString.split('\n'): #split Data \n
        for data_step2 in data_step1.split('.'): #split Data .
            splitedData.append(data_step2.split(":"))
    
    return [splitedData, rawDataString]

def formatData_Json_세부특기사항(raw_data_list):
    nomalizationData = dataNormalization_세부특기사항(raw_data_list)
    raw_data_list = nomalizationData[0]
    formatedData = {}
    dataIndex = ''
    
    
    for index in raw_data_list:
       
        if len(index) > 1: # 키 데이터 분리, 키 지정
            dataIndex = index[0].rstrip()
            formatedData[dataIndex] = ''
            for cursor in index[1:]: formatedData[dataIndex] += cursor #데이터 기준
            
        else :
            for cursor in index: formatedData[dataIndex] += cursor
        
        if len(index) == 1:
            if index[0] == '': #끝이. 이었던 부분은 데이터 split시 '' 혹은 ' '형태로 나타남
                formatedData[dataIndex] += '.'
            else:
                try: # 날짜 데이터 형식은 YYYY(길이 4), MM, DD(길이 2)로 나타남 날짜 데이터 나타날경우 . 붙여줌
                    if len(index[0]) == 2 or len(index[0]) == 4:
                        int(index[0]) #날짜 데이터 복원 여기서 날짜 형식이 아닌경우 에러남
                        formatedData[dataIndex] += '.' #데이터에 따라 제대로 작동되지 않을 수 있음으로 날짜 . 붙이는건 포기 추천
                except:
                    pass
        
    return {"formatedData": formatedData, "rawData": nomalizationData[1]}
            
        
        
rawData = getRawData("./testDoc1.pdf") #봉사실적, 창의적체험활동상황, 진로희망사항, 세부특기사항, 수상실적, 과목


file = open("ouputData.txt", 'w')

ouputData = {
    "봉사실적": formatData_Json_봉사실적(rawData[0]),
    "창의적체험활동상황": formatData_Json_창의적체험활동상황(rawData[1]),
    "진로희망사항": formatData_Json_진로희망사항(rawData[2]),
    "세부특기사항": formatData_Json_세부특기사항(rawData[3]),
    "수상실적": formatData_Json_수상실적(rawData[4]),
}

file.write(str(ouputData))





2019


({'formatedData': {'국어Ⅱ': " 모둠별 토론 수업을 통해 '사람을 무는 개를 안락사 시켜야 하는가'라는 주제를 직접 선택, 모둠을 이끄는 모둠장이 되어 적극적으로 토론 활동을 준비하는 등 열정있는 모습을 보여줌 특히 찬성측 토론자로서  모둠원간의 팀워크가 중요하다는 점을 깨닫고 친구들을 참여하도록 독려하는 리더십있는 모습을 강하게 보여주는 등 자신이 흥미로워하는 분야에 대한 적극적인 모습이 남다름 이 활동을 통해 신뢰성 있고 타당하고 공정성 있는 자료를 찾는 것이 생각보다 힘들다는 것을 몸소 느끼는 등 문제해결적 사고력을 배양함 찬반 양측의 자료를 찾으면서 관점에 따라 자료의해석이 쉽게 달라질 수 있다는 것에 대해 알게 됨 무엇보다 토론을 통해 사회를 바라볼 때 다양한 관점에 따라 판단해야 한다는 것을 다시금 되새김 더 나아가 더 나은 해결책에 대해 궁구해 봄으로써 깊이있는 사고를 보여줌 또한 한글의 음소문자가 컴퓨터 키보드 자판 입력의 편리성과 관련하여 프리젠테이션을 준비하여 친구들 앞에서 발표하였음 ",
   '수학Ⅱ': ' 수열 단원 수업 중 다른 학생들이 꺼려하는 문제도 스스럼없이 나와 풀이를 하였음 본인의 풀이와 관련하여부족한 점을 알려 주면 이를 극복하기 위해 노력을 기울여 개선하려는 의욕을 보임 자신의 진로 희망인 프로그래밍과관련해 연관성이 높은 집합, 수열 단원에 관심을 보이며 관련 단원의 내용을 정리하고 자료를 모으며 프로그래밍과 관련하여 재해석하기도 하고, 자신의 전공을 접목시켜 아포피시스라는 프랙탈아트 프로그램을 익혀 학교에서 주최하는수학경진대회에 참여하기도 하는 모습을 보이는 등 수학 교과의 내용을 이해하고 자신의 진로 희망분야에 활용하려는노력이 돋보임.',
   '실용영어독해와작문': ' 해커톤(Higthon show me the coding & SNU Hakerthon)의 프로그래머로써 참여하여, 영문보고서를 제출하였음.평소 관심영역에 해당하는 과학관련 지문을 읽고 해당 분야에 대한 추가적인 발표를 목표어(Target

In [955]:
index = ['1', '독서', '교사', '교사', '가르치고 이해시키고 깨닫게 하\n는 것에 보람을 느낌.']
print(index[0:-1])

['1', '독서', '교사', '교사']


In [986]:
data = {"a": []}
print["a"]

TypeError: 'builtin_function_or_method' object is not subscriptable

In [1044]:
data = [None, '2014.08.21.', '-', '2014.12.10.', '(학교)성보고등학교', '휴대폰 관리', '8', '12']

data = 


print(data)

[None, '2014.12.10.', '(학교)성보고등학교', '휴대폰 관리', '8', '12']


In [1367]:
"DDDDDD".split(":")

['DDDDDD']